# 🎵 Harmonix Batch Import - YouTube Playlist to Stems

Downloads songs from YouTube and separates into **6 stems**:
- 🎤 **vocals** - Isolated vocals
- 🥁 **drums** - Drum track
- 🎸 **bass** - Bass track  
- 🎹 **other** - Piano, synths, guitars, etc.
- 🎵 **instrumental** - Everything minus vocals (for karaoke)
- 🎧 **original** - Original audio file

**Instructions:**
1. **Runtime → Change runtime type → T4 GPU**
2. Run all cells in order
3. Authorize Google Drive when prompted
4. Enter playlist URL and let it run!
5. Find stems in **My Drive/Harmonix_Stems/**

In [ ]:
#@title 1️⃣ Install Dependencies & Mount Google Drive
print("📦 Installing dependencies...")
!pip install -q demucs yt-dlp torch torchaudio
!apt-get -qq install ffmpeg
print("✅ Dependencies installed!")

print("\n💾 Mounting Google Drive...")
from google.colab import drive
drive.mount('/content/drive')
print("✅ Google Drive mounted!")

import torch
if torch.cuda.is_available():
    print(f"\n🚀 GPU: {torch.cuda.get_device_name(0)} ({torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB)")
else:
    print("\n⚠️ No GPU! Go to Runtime → Change runtime type → T4 GPU")

In [ ]:
#@title 2️⃣ Setup Functions
import subprocess
import json
import os
import shutil
import numpy as np
from pathlib import Path

DOWNLOAD_DIR = Path("/content/downloads")
DRIVE_OUTPUT = Path("/content/drive/MyDrive/Harmonix_Stems")
DOWNLOAD_DIR.mkdir(exist_ok=True)
DRIVE_OUTPUT.mkdir(exist_ok=True)

print(f"💾 Stems will be saved to: My Drive/Harmonix_Stems/")

def parse_playlist(url):
    print(f"🔍 Parsing: {url[:60]}...")
    result = subprocess.run(['yt-dlp', '--flat-playlist', '-J', '--no-warnings', url], capture_output=True, text=True)
    if result.returncode != 0:
        result = subprocess.run(['yt-dlp', '-J', '--no-warnings', url], capture_output=True, text=True)
        if result.returncode == 0:
            data = json.loads(result.stdout)
            return [{'id': data['id'], 'title': data['title'], 'duration': data.get('duration', 0)}]
        raise Exception(f"Failed: {result.stderr[:100]}")
    data = json.loads(result.stdout)
    videos = [{'id': e.get('id',''), 'title': e.get('title','Unknown'), 'duration': e.get('duration',0)} for e in data.get('entries', [data]) if e]
    print(f"✅ Found {len(videos)} videos")
    return videos

def safe_name(title):
    return "".join(c for c in title if c.isalnum() or c in ' -_').strip()[:50]

def download_audio(video_id, title):
    name = safe_name(title)
    output_path = DOWNLOAD_DIR / f"{name}.mp3"
    if output_path.exists():
        return output_path
    print(f"   📥 Downloading...")
    subprocess.run(['yt-dlp', '-x', '--audio-format', 'mp3', '--audio-quality', '0', '-o', str(output_path), '--no-playlist', '--no-warnings', f'https://www.youtube.com/watch?v={video_id}'], capture_output=True)
    if output_path.exists():
        return output_path
    alt = DOWNLOAD_DIR / f"{name}.mp3.mp3"
    if alt.exists():
        alt.rename(output_path)
        return output_path
    raise Exception("Download failed")

def separate_stems(audio_path, song_name):
    """Separate into all stems + create instrumental"""
    song_folder = DRIVE_OUTPUT / song_name
    
    # Check if already done (has all 6 files)
    if song_folder.exists() and len(list(song_folder.glob("*.mp3"))) >= 5:
        print(f"   ⏭️ Already processed")
        return song_folder
    
    print(f"   🎵 Separating 4 stems (GPU)...")
    temp_out = Path("/content/temp_stems")
    
    # Run Demucs with htdemucs_ft model (vocals, drums, bass, other)
    result = subprocess.run([
        'python', '-m', 'demucs',
        '-n', 'htdemucs_ft',
        '-o', str(temp_out),
        '--mp3', '--mp3-bitrate', '320',
        str(audio_path)
    ], capture_output=True, text=True)
    
    if result.returncode != 0:
        print(f"   ⚠️ Error: {result.stderr[:100]}")
        return None
    
    song_folder.mkdir(exist_ok=True)
    demucs_out = temp_out / 'htdemucs_ft' / audio_path.stem
    
    if demucs_out.exists():
        # Copy stems with proper naming: songname_stemtype.mp3
        stem_map = {'vocals': 'vocals', 'drums': 'drums', 'bass': 'bass', 'other': 'other'}
        
        for stem_file in demucs_out.glob("*.mp3"):
            stem_type = stem_file.stem
            if stem_type in stem_map:
                dest = song_folder / f"{song_name}_{stem_map[stem_type]}.mp3"
                shutil.copy2(str(stem_file), str(dest))
        
        # Create instrumental (drums + bass + other)
        print(f"   🎹 Creating instrumental...")
        drums = demucs_out / "drums.mp3"
        bass = demucs_out / "bass.mp3"
        other = demucs_out / "other.mp3"
        instrumental_path = song_folder / f"{song_name}_instrumental.mp3"
        
        if drums.exists() and bass.exists() and other.exists():
            # Use ffmpeg to mix drums + bass + other
            subprocess.run([
                'ffmpeg', '-y', '-i', str(drums), '-i', str(bass), '-i', str(other),
                '-filter_complex', 'amix=inputs=3:duration=longest',
                '-b:a', '320k', str(instrumental_path)
            ], capture_output=True)
        
        # Copy original
        original_dest = song_folder / f"{song_name}_original.mp3"
        shutil.copy2(str(audio_path), str(original_dest))
        
        shutil.rmtree(temp_out, ignore_errors=True)
    
    stems = list(song_folder.glob("*.mp3"))
    print(f"   💾 Saved {len(stems)} files to Drive!")
    return song_folder

print("✅ Functions ready!")

In [ ]:
#@title 3️⃣ Process Playlist
#@markdown ### Paste YouTube playlist or video URL:
PLAYLIST_URL = "https://www.youtube.com/playlist?list=YOUR_PLAYLIST_ID" #@param {type:"string"}
MAX_SONGS = 10 #@param {type:"slider", min:1, max:100, step:1}

print("="*60)
print("🎵 HARMONIX BATCH IMPORT → GOOGLE DRIVE")
print("   Stems: vocals, drums, bass, other, instrumental, original")
print("="*60)

videos = parse_playlist(PLAYLIST_URL)[:MAX_SONGS]
print(f"\n📋 Processing {len(videos)} songs:\n")
for i, v in enumerate(videos, 1):
    dur = f"{v['duration']//60}:{v['duration']%60:02d}" if v.get('duration') else "?:??"
    print(f"   {i}. {v['title'][:45]} ({dur})")
print("\n" + "="*60)

completed, failed = 0, []
for i, video in enumerate(videos, 1):
    print(f"\n[{i}/{len(videos)}] {video['title'][:40]}")
    try:
        audio = download_audio(video['id'], video['title'])
        name = safe_name(video['title'])
        if separate_stems(audio, name):
            completed += 1
        else:
            failed.append(video['title'][:30])
    except Exception as e:
        print(f"   ❌ {e}")
        failed.append(video['title'][:30])

print("\n" + "="*60)
print(f"🎉 DONE! {completed}/{len(videos)} processed")
print(f"💾 Location: My Drive/Harmonix_Stems/")
if failed:
    print(f"❌ Failed: {failed}")

In [ ]:
#@title 📂 View Stems
print("📂 Harmonix_Stems/\n")
for d in sorted(DRIVE_OUTPUT.iterdir()):
    if d.is_dir():
        files = list(d.glob("*.mp3"))
        print(f"🎵 {d.name}/ ({len(files)} stems)")
        for f in sorted(files):
            print(f"   └─ {f.name}")

In [ ]:
#@title 🧹 Cleanup Cache
import shutil
for p in [DOWNLOAD_DIR, Path("/content/temp_stems")]:
    if p.exists(): shutil.rmtree(p)
print("✅ Cache cleared! Stems safe in Drive.")

# 🎵 Harmonix Batch Import - YouTube Playlist to Stems

This notebook downloads songs from a YouTube playlist and separates them into stems (vocals, instrumental) using Demucs AI.

**Features:**
- ⚡ GPU-accelerated stem separation (much faster than CPU!)
- 📥 Download entire YouTube playlists
- 🎤 Karaoke-ready vocals + instrumental stems
- 💾 Saves directly to Google Drive

**Instructions:**
1. Go to **Runtime → Change runtime type → T4 GPU**
2. Run all cells in order
3. Authorize Google Drive access when prompted
4. Enter your playlist URL and let it run!
5. Find your stems in **My Drive/Harmonix_Stems/**

In [ ]:
#@title 1️⃣ Install Dependencies & Mount Google Drive
print("📦 Installing dependencies...")
!pip install -q demucs yt-dlp torch torchaudio
!apt-get -qq install ffmpeg
print("✅ Dependencies installed!")

# Mount Google Drive
print("\n💾 Mounting Google Drive...")
from google.colab import drive
drive.mount('/content/drive')
print("✅ Google Drive mounted!")

# Check GPU
import torch
if torch.cuda.is_available():
    print(f"\n🚀 GPU detected: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("\n⚠️ No GPU! Go to Runtime → Change runtime type → T4 GPU")

In [ ]:
#@title 2️⃣ Setup Functions
import subprocess
import json
import os
import shutil
from pathlib import Path
from datetime import datetime

# Directories
DOWNLOAD_DIR = Path("/content/downloads")
DRIVE_OUTPUT = Path("/content/drive/MyDrive/Harmonix_Stems")
DOWNLOAD_DIR.mkdir(exist_ok=True)
DRIVE_OUTPUT.mkdir(exist_ok=True)

print(f"💾 Stems will be saved to: {DRIVE_OUTPUT}")

def parse_playlist(url):
    print(f"🔍 Parsing playlist: {url}")
    cmd = ['yt-dlp', '--flat-playlist', '-J', '--no-warnings', url]
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    if result.returncode != 0:
        cmd_single = ['yt-dlp', '-J', '--no-warnings', url]
        result = subprocess.run(cmd_single, capture_output=True, text=True)
        if result.returncode == 0:
            data = json.loads(result.stdout)
            return [{'id': data['id'], 'title': data['title'], 'duration': data.get('duration', 0)}]
        raise Exception(f"Failed to parse: {result.stderr}")
    
    data = json.loads(result.stdout)
    videos = []
    for entry in data.get('entries', [data]):
        if entry:
            videos.append({'id': entry.get('id', ''), 'title': entry.get('title', 'Unknown'), 'duration': entry.get('duration', 0)})
    print(f"✅ Found {len(videos)} videos")
    return videos

def download_audio(video_id, title):
    safe_title = "".join(c for c in title if c.isalnum() or c in ' -_').strip()[:50]
    output_path = DOWNLOAD_DIR / f"{safe_title}.mp3"
    if output_path.exists():
        print(f"   ⏭️ Already downloaded")
        return output_path
    print(f"   📥 Downloading...")
    cmd = ['yt-dlp', '-x', '--audio-format', 'mp3', '--audio-quality', '0', '-o', str(output_path), '--no-playlist', '--no-warnings', f'https://www.youtube.com/watch?v={video_id}']
    subprocess.run(cmd, capture_output=True, text=True)
    if output_path.exists():
        return output_path
    alt_path = DOWNLOAD_DIR / f"{safe_title}.mp3.mp3"
    if alt_path.exists():
        alt_path.rename(output_path)
        return output_path
    raise Exception("Download failed")

def separate_and_save(audio_path, song_name):
    song_folder = DRIVE_OUTPUT / song_name
    if song_folder.exists() and any(song_folder.glob("*.mp3")):
        print(f"   ⏭️ Already in Drive")
        return song_folder
    print(f"   🎵 Separating stems (GPU)...")
    temp_out = Path("/content/temp_stems")
    cmd = ['python', '-m', 'demucs', '-n', 'htdemucs_ft', '--two-stems', 'vocals', '-o', str(temp_out), '--mp3', '--mp3-bitrate', '320', str(audio_path)]
    result = subprocess.run(cmd, capture_output=True, text=True)
    if result.returncode != 0:
        print(f"   ⚠️ Error: {result.stderr[:100]}")
        return None
    song_folder.mkdir(exist_ok=True)
    demucs_out = temp_out / 'htdemucs_ft' / audio_path.stem
    if demucs_out.exists():
        for stem_file in demucs_out.glob("*.mp3"):
            stem_name = 'instrumental' if stem_file.stem == 'no_vocals' else stem_file.stem
            shutil.copy2(str(stem_file), str(song_folder / f"{song_name}_{stem_name}.mp3"))
        shutil.rmtree(temp_out, ignore_errors=True)
    print(f"   💾 Saved to Drive!")
    return song_folder

print("✅ Functions loaded!")

In [ ]:
#@title 3️⃣ Enter Playlist URL and Process
#@markdown ### Paste your YouTube playlist or video URL:
PLAYLIST_URL = "https://www.youtube.com/playlist?list=YOUR_PLAYLIST_ID" #@param {type:"string"}
MAX_SONGS = 10 #@param {type:"slider", min:1, max:100, step:1}

print("="*60)
print("🎵 HARMONIX BATCH IMPORT → GOOGLE DRIVE")
print("="*60)

videos = parse_playlist(PLAYLIST_URL)[:MAX_SONGS]
print(f"\n📋 Processing {len(videos)} songs:\n")
for i, v in enumerate(videos, 1):
    dur = f"{v['duration']//60}:{v['duration']%60:02d}" if v.get('duration') else "?:??"
    print(f"   {i}. {v['title'][:45]}... ({dur})")
print(f"\n💾 Output: My Drive/Harmonix_Stems/")
print("="*60)

completed, failed = 0, []
for i, video in enumerate(videos, 1):
    print(f"\n[{i}/{len(videos)}] {video['title'][:40]}")
    try:
        audio_path = download_audio(video['id'], video['title'])
        safe_name = "".join(c for c in video['title'] if c.isalnum() or c in ' -_').strip()[:50]
        if separate_and_save(audio_path, safe_name):
            completed += 1
        else:
            failed.append(video['title'][:40])
    except Exception as e:
        print(f"   ❌ Error: {e}")
        failed.append(video['title'][:40])

print("\n" + "="*60)
print("🎉 COMPLETE!")
print(f"   ✅ Processed: {completed}/{len(videos)}")
print(f"   💾 Location: My Drive/Harmonix_Stems/")
if failed:
    print(f"   ❌ Failed: {len(failed)}")

In [ ]:
#@title 📂 View Stems in Drive
print("📂 Stems in Google Drive:\n")
total = 0
for d in sorted(DRIVE_OUTPUT.iterdir()):
    if d.is_dir():
        total += 1
        print(f"🎵 {d.name}/")
        for f in d.glob("*.mp3"):
            print(f"   └─ {f.name}")
print(f"\n📊 Total: {total} songs")

In [ ]:
#@title 🧹 Cleanup Cache
import shutil
for p in [DOWNLOAD_DIR, Path("/content/temp_stems")]:
    if p.exists(): shutil.rmtree(p)
print("✅ Cache cleared! Stems safe in Drive.")

# 🎵 Harmonix Batch Import - YouTube Playlist to Stems

This notebook downloads songs from a YouTube playlist and separates them into stems (vocals, instrumental) using Demucs AI.

**Features:**
- ⚡ GPU-accelerated stem separation (much faster than CPU!)
- 📥 Download entire YouTube playlists
- 🎤 Karaoke-ready vocals + instrumental stems
- 📦 Download results as a ZIP file

**Instructions:**
1. Go to **Runtime → Change runtime type → T4 GPU**
2. Run all cells in order
3. Enter your playlist URL when prompted
4. Download the ZIP when complete!

In [ ]:
#@title 1️⃣ Install Dependencies (Run once)
print("📦 Installing dependencies...")
!pip install -q demucs yt-dlp torch torchaudio
!apt-get -qq install ffmpeg
print("✅ All dependencies installed!")

# Check GPU
import torch
if torch.cuda.is_available():
    print(f"🚀 GPU detected: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("⚠️ No GPU detected! Go to Runtime → Change runtime type → T4 GPU")

In [ ]:
#@title 2️⃣ Setup Functions
import subprocess
import json
import os
import shutil
from pathlib import Path
from google.colab import files
import zipfile
from datetime import datetime

# Create directories
DOWNLOAD_DIR = Path("/content/downloads")
OUTPUT_DIR = Path("/content/stems")
DOWNLOAD_DIR.mkdir(exist_ok=True)
OUTPUT_DIR.mkdir(exist_ok=True)

def parse_playlist(url):
    """Parse YouTube playlist and return list of videos"""
    print(f"🔍 Parsing playlist: {url}")
    
    cmd = [
        'yt-dlp',
        '--flat-playlist',
        '-J',
        '--no-warnings',
        url
    ]
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    if result.returncode != 0:
        # Try as single video
        cmd_single = ['yt-dlp', '-J', '--no-warnings', url]
        result = subprocess.run(cmd_single, capture_output=True, text=True)
        if result.returncode == 0:
            data = json.loads(result.stdout)
            return [{'id': data['id'], 'title': data['title'], 'duration': data.get('duration', 0)}]
        raise Exception(f"Failed to parse: {result.stderr}")
    
    data = json.loads(result.stdout)
    videos = []
    
    entries = data.get('entries', [data])
    for entry in entries:
        if entry:
            videos.append({
                'id': entry.get('id', ''),
                'title': entry.get('title', 'Unknown'),
                'duration': entry.get('duration', 0)
            })
    
    print(f"✅ Found {len(videos)} videos")
    return videos

def download_audio(video_id, title):
    """Download audio from YouTube video"""
    safe_title = "".join(c for c in title if c.isalnum() or c in ' -_').strip()[:50]
    output_path = DOWNLOAD_DIR / f"{safe_title}.mp3"
    
    if output_path.exists():
        print(f"   ⏭️ Already downloaded: {safe_title}")
        return output_path
    
    print(f"   📥 Downloading: {safe_title}...")
    
    cmd = [
        'yt-dlp',
        '-x',
        '--audio-format', 'mp3',
        '--audio-quality', '0',
        '-o', str(output_path),
        '--no-playlist',
        '--no-warnings',
        f'https://www.youtube.com/watch?v={video_id}'
    ]
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    if output_path.exists():
        return output_path
    
    # Check for .mp3.mp3 issue
    alt_path = DOWNLOAD_DIR / f"{safe_title}.mp3.mp3"
    if alt_path.exists():
        alt_path.rename(output_path)
        return output_path
    
    raise Exception(f"Download failed: {result.stderr}")

def separate_stems(audio_path, output_name):
    """Separate audio into stems using Demucs"""
    stem_dir = OUTPUT_DIR / output_name
    
    if stem_dir.exists() and any(stem_dir.glob("*.mp3")):
        print(f"   ⏭️ Already processed: {output_name}")
        return stem_dir
    
    stem_dir.mkdir(exist_ok=True)
    print(f"   🎵 Separating stems...")
    
    # Run Demucs with GPU
    cmd = [
        'python', '-m', 'demucs',
        '-n', 'htdemucs_ft',
        '--two-stems', 'vocals',  # Karaoke mode: vocals + instrumental
        '-o', str(OUTPUT_DIR / 'temp'),
        '--mp3',
        '--mp3-bitrate', '320',
        str(audio_path)
    ]
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    if result.returncode != 0:
        print(f"   ⚠️ Demucs error: {result.stderr[:200]}")
        return None
    
    # Move stems to output folder
    temp_dir = OUTPUT_DIR / 'temp' / 'htdemucs_ft' / audio_path.stem
    if temp_dir.exists():
        for stem_file in temp_dir.glob("*.mp3"):
            stem_name = stem_file.stem  # vocals or no_vocals
            if stem_name == 'no_vocals':
                stem_name = 'instrumental'
            dest = stem_dir / f"{output_name}_{stem_name}.mp3"
            shutil.move(str(stem_file), str(dest))
        shutil.rmtree(temp_dir, ignore_errors=True)
    
    print(f"   ✅ Stems saved!")
    return stem_dir

def create_zip():
    """Create ZIP of all stems"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    zip_path = f"/content/harmonix_stems_{timestamp}.zip"
    
    print("\n📦 Creating ZIP file...")
    
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zf:
        for stem_dir in OUTPUT_DIR.iterdir():
            if stem_dir.is_dir() and stem_dir.name != 'temp':
                for mp3_file in stem_dir.glob("*.mp3"):
                    arcname = f"{stem_dir.name}/{mp3_file.name}"
                    zf.write(mp3_file, arcname)
    
    size_mb = os.path.getsize(zip_path) / (1024 * 1024)
    print(f"✅ ZIP created: {size_mb:.1f} MB")
    
    return zip_path

print("✅ Functions loaded!")

In [ ]:
#@title 3️⃣ Enter Playlist URL and Process
#@markdown Paste your YouTube playlist or video URL below:

PLAYLIST_URL = "https://www.youtube.com/playlist?list=PLxxxxxxxx" #@param {type:"string"}
MAX_SONGS = 10 #@param {type:"slider", min:1, max:50, step:1}

print("="*60)
print("🎵 HARMONIX BATCH IMPORT")
print("="*60)

# Parse playlist
videos = parse_playlist(PLAYLIST_URL)

# Limit to max songs
videos = videos[:MAX_SONGS]
print(f"\n📋 Processing {len(videos)} songs:\n")
for i, v in enumerate(videos, 1):
    duration = f"{v['duration']//60}:{v['duration']%60:02d}" if v['duration'] else "?:??"
    print(f"   {i}. {v['title'][:50]} ({duration})")

print("\n" + "="*60)

# Process each video
completed = 0
failed = []

for i, video in enumerate(videos, 1):
    print(f"\n[{i}/{len(videos)}] {video['title'][:40]}...")
    
    try:
        # Download
        audio_path = download_audio(video['id'], video['title'])
        
        # Create safe name for output folder
        safe_name = "".join(c for c in video['title'] if c.isalnum() or c in ' -_').strip()[:50]
        
        # Separate stems
        stem_dir = separate_stems(audio_path, safe_name)
        
        if stem_dir:
            completed += 1
        else:
            failed.append(video['title'])
            
    except Exception as e:
        print(f"   ❌ Error: {e}")
        failed.append(video['title'])

print("\n" + "="*60)
print(f"\n🎉 COMPLETE!")
print(f"   ✅ Processed: {completed}/{len(videos)}")
if failed:
    print(f"   ❌ Failed: {len(failed)}")
    for f in failed:
        print(f"      - {f[:40]}")

In [ ]:
#@title 4️⃣ Download Results as ZIP

# Create and download ZIP
zip_path = create_zip()

print("\n📥 Starting download...")
files.download(zip_path)

print("\n✅ Download started! Check your browser downloads.")

In [ ]:
#@title 🔧 Optional: View Generated Stems

print("📁 Generated Stems:\n")

for stem_dir in sorted(OUTPUT_DIR.iterdir()):
    if stem_dir.is_dir() and stem_dir.name != 'temp':
        print(f"📂 {stem_dir.name}/")
        for mp3_file in sorted(stem_dir.glob("*.mp3")):
            size_mb = mp3_file.stat().st_size / (1024 * 1024)
            print(f"   🎵 {mp3_file.name} ({size_mb:.1f} MB)")
        print()

In [ ]:
#@title 🧹 Optional: Clean Up (Free disk space)

import shutil

print("🧹 Cleaning up...")

# Remove downloads
if DOWNLOAD_DIR.exists():
    shutil.rmtree(DOWNLOAD_DIR)
    print("   ✅ Removed downloads folder")

# Remove stems
if OUTPUT_DIR.exists():
    shutil.rmtree(OUTPUT_DIR)
    print("   ✅ Removed stems folder")

# Remove any ZIP files
for zf in Path("/content").glob("harmonix_stems_*.zip"):
    zf.unlink()
    print(f"   ✅ Removed {zf.name}")

print("\n✅ Cleanup complete!")